In [4]:
import requests # pip install requests
from consumer_details import CONSUMER_KEY, CONSUMER_SECRET, USERNAME, PASSWORD
import pandas as pd # pip install pandas

# Generate Access Token
def generate_token():    
    payload = {
        'grant_type': 'password',
        'client_id': CONSUMER_KEY,
        'client_secret': CONSUMER_SECRET,
        'username': USERNAME,
        'password': PASSWORD
    }
    oauth_endpoint = '/services/oauth2/token'
    response = requests.post(DOMAIN + oauth_endpoint, data=payload)
    return response.json()

DOMAIN = 'https://dir0000007kzc2ai-dev-ed.develop.my.salesforce.com'
access_token = generate_token()['access_token']
headers = {
    'Authorization': 'Bearer ' + access_token
}
# print(access_token)
# Example 1. Run a SOQL query
def query(soql_query):
    try:
        # soql_query = 'SELECT name FROM opportunity'
        endpoint = '/services/data/v54.0/query/'
        records = []
        response = requests.get(DOMAIN + endpoint, headers=headers, params={'q': soql_query})
        total_size = response.json()['totalSize']
        records.extend(response.json()['records'])

        while not response.json()['done']:
            response = requests.get(DOMAIN + endpoint + response.json()['nextRecordsUrl'], headers=headers)
            records.extend(response.json()['records'])
        return {'record_size': total_size, 'records': records}
    except Exception as e:
        print(e)
        return

records = query('SELECT id, name, type FROM opportunity')
print(records)
df = pd.DataFrame(records['records'])
print(df)

# Example 2. Retrieve an object's metadata
def retrieve_object_metadata(object_api_name):
    response = requests.get(DOMAIN + f'/services/data/v56.0/sobjects/{object_api_name}/describe', headers=headers)
    return response.json()

# object_id = 'account'
object_id = 'opportunity'
object_metadata = retrieve_object_metadata(object_id)
print(object_metadata)
df_metadata = pd.DataFrame(object_metadata['fields'])
df_metadata.to_csv('account metadata information.csv', index=False)

{'record_size': 31, 'records': [{'attributes': {'type': 'Opportunity', 'url': '/services/data/v54.0/sobjects/Opportunity/006IR00001Tj8t3YAB'}, 'Id': '006IR00001Tj8t3YAB', 'Name': 'Dickenson Mobile Generators', 'Type': 'New Customer'}, {'attributes': {'type': 'Opportunity', 'url': '/services/data/v54.0/sobjects/Opportunity/006IR00001Tj8t5YAB'}, 'Id': '006IR00001Tj8t5YAB', 'Name': 'United Oil Office Portable Generators', 'Type': 'Existing Customer - Upgrade'}, {'attributes': {'type': 'Opportunity', 'url': '/services/data/v54.0/sobjects/Opportunity/006IR00001Tj8t8YAB'}, 'Id': '006IR00001Tj8t8YAB', 'Name': 'Express Logistics Standby Generator', 'Type': 'New Customer'}, {'attributes': {'type': 'Opportunity', 'url': '/services/data/v54.0/sobjects/Opportunity/006IR00001Tj8t6YAB'}, 'Id': '006IR00001Tj8t6YAB', 'Name': 'GenePoint Standby Generator', 'Type': 'New Customer'}, {'attributes': {'type': 'Opportunity', 'url': '/services/data/v54.0/sobjects/Opportunity/006IR00001Tj8t9YAB'}, 'Id': '006IR

PermissionError: [Errno 13] Permission denied: 'account metadata information.csv'